In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Part_Number'].tolist()

总数量：2


['3310714', '3310721']

In [3]:
crawler_status = 'error'

crawler_status

'error'

In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get('https://apdi.autocaredata.com/items/' + input_.loc[a, 'Part_Number'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<!DOCTYPE html><html><head><title>Apdi MyCarParts</title><meta name="csrf-param" content="authenticity_token" />\n<meta name="csrf-token" content="7JnxKwSozG4HDHnA8JDyk6rG1IyB1iXrxP8DqnF1DRtifhred87UphZVr9VCjnA2LdThqtwveQcVUmTJVMEYGw==" /><link rel="stylesheet" media="all" href="/assets/application-384dabf928e2a401f55039bf9f70b1302fe613093561e5de6adf83003230666e.css" data-turbolinks-track="reload" /><script src="/assets/application-1e10bd04301545a2277bed24e0b59011d911150f243196116b5950e5f9d4b1fa.js" data-turbolinks-track="reload"></script></head><body><div class="bg-dark" id="header"><div class="container"><nav class="navbar navbar-expand-lg navbar-dark bg-dark row"><a class="navbar-brand" href="http://apdi.autocaredata.com"><img style="position:absolute; top: 0; left:1rem; max-height: 100px" src="https://res.cloudinary.com/illumaware/image/upload/v1661918761/agilitysm_x7dtuh.png" alt="Agilitysm x7dtuh" /></a><button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle n

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

In [7]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./info.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Apdi MyCarParts
  </title>
  <meta content="authenticity_token" name="csrf-param"/>
  <meta content="7JnxKwSozG4HDHnA8JDyk6rG1IyB1iXrxP8DqnF1DRtifhred87UphZVr9VCjnA2LdThqtwveQcVUmTJVMEYGw==" name="csrf-token"/>
  <link data-turbolinks-track="reload" href="/assets/application-384dabf928e2a401f55039bf9f70b1302fe613093561e5de6adf83003230666e.css" media="all" rel="stylesheet"/>
  <script data-turbolinks-track="reload" src="/assets/application-1e10bd04301545a2277bed24e0b59011d911150f243196116b5950e5f9d4b1fa.js">
  </script>
 </head>
 <body>
  <div class="bg-dark" id="header">
   <div class="container">
    <nav class="navbar navbar-expand-lg navbar-dark bg-dark row">
     <a class="navbar-brand" href="http://apdi.autocaredata.com">
      <img alt="Agilitysm x7dtuh" src="https://res.cloudinary.com/illumaware/image/upload/v1661918761/agilitysm_x7dtuh.png" style="position:absolute; top: 0; left:1rem; max-height: 100px"/>
     </a>
     <button aria-c

In [9]:
if len(html.xpath('//div[@class="carousel-item active"]/div[@class="image"]/img/@src')) == 0:
    src = ''
else:
    src = html.xpath('//div[@class="carousel-item active"]/div[@class="image"]/img/@src')[0].strip()

src

[]

In [ ]:
list_tr = html.xpath('//div[@id="collapseBuyersGuideDetail"]/table/tbody/tr')

list_tr

In [ ]:
list_vehicle = []
for tr in list_tr:
    make = tr.xpath('./td[1]/text()')[0].strip()
    model = tr.xpath('./td[2]/text()')[0].strip()
    engine = tr.xpath('./td[3]/ul/li[2]/text()')[0].strip()
    year = tr.xpath('./td[4]/text()')[0].strip()
    
    list_vehicle.append(make + ' ' + model + ' ' + year + ' ' + engine)
    
vehicle = '\n'.join(list_vehicle)

print(vehicle)

In [ ]:
list_tr = html.xpath('//div[@id="collapseInterchanges"]/table/tbody/tr')

list_tr

In [ ]:
dict_oe = {}
for tr in list_tr:
    key = tr.xpath('./td[2]/text()')[0].strip()
    
    if key not in dict_oe:
        dict_oe[key] = [tr.xpath('./td[1]/text()')[0].strip()]
    else:
        dict_oe[key].append(tr.xpath('./td[1]/text()')[0].strip())
        
    dict_oe[key] = sorted(list(set(dict_oe[key])))
        
dict_oe

In [ ]:
list_tr = html.xpath('//div[@class="item_attributes"]/table/tr/td/table/tr')

list_tr

In [ ]:
df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                         'Vehicle': vehicle,
                         'Src': src}])

for key in dict_oe:
    df_temp.loc[0, 'OE_' + key] = ';'.join(dict_oe[key])

for tr in list_tr:
    df_temp.loc[0, tr.xpath('./td/text()')[0].strip()[:-1]] = tr.xpath('./th/text()')[0].strip()

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

In [ ]:
crawler_status = 'ok'

crawler_status

In [ ]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

In [ ]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_info.xlsx', index=False)

output_correct

In [ ]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_ifno_error.xlsx', index=False)

output_error